# **Programming Assessment \#4**

Names: ABERIN, Shawn,   LIM, Kyle,  SINGSON, Raymond

More information on the assessment is found in our Canvas course.

# **Load Data**

*While you don't have to separate your code into blocks, it might be easier if you separated loading your data from actually implementation of your code. Consider placing all loading of data into the code block below.*

In [84]:
# The code is defining a function called `make_p` that takes a DataFrame `df` as input.
# lang must contain a column 'count' that contains number types
#load error data

import pandas as pd
def make_p(pd):
    total = pd['count'].sum()
    inv = 1/total
    pd['P'] = pd['count']*inv
    return pd
data = []
# Trust taht python will open it correctly.
with open("count_1edit.txt", "r") as f:
    x = f.read().splitlines()
    gen = (dat.split('\t') for dat in x)
    data = [(rec[0], int(rec[1])) for rec in gen if len(rec) == 2]

err = pd.DataFrame.from_records(data, columns=['error', 'count'])
err = make_p(err)

print(err)


      error  count         P
0       e|i    917  0.023469
1       a|e    856  0.021908
2       i|e    771  0.019732
3       e|a    749  0.019169
4       a|i    559  0.014307
...     ...    ...       ...
1582     |c      1  0.000026
1583     |a      1  0.000026
1584     |'      1  0.000026
1585    w|       1  0.000026
1586   t|t       1  0.000026

[1587 rows x 3 columns]


In [85]:
#loading corpus files
import nltk
nltk.download('gutenberg')
# nltk.corpus.gutenberg.fileids()

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\shawn\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [86]:
from collections import Counter
from nltk.probability import FreqDist
import string
import time
from statistics import mean 
print("Extracting all documents from NLTK's Project Gutenberg Collection...")
all_words = Counter()

for filename in nltk.corpus.gutenberg.fileids():
    word = (word.lower() for word in nltk.corpus.gutenberg.words(filename) if word.isalpha())
    all_words.update(word)


lang = pd.DataFrame.from_dict(all_words, orient='index').reset_index()
# print(mean(t),lang)

Extracting all documents from NLTK's Project Gutenberg Collection...


In [87]:
lang = lang.rename(columns={'index':'word', 0:'count'})
lang = make_p(lang)

print(lang)

           word  count             P
0          emma    866  4.055446e-04
1            by   8512  3.986138e-03
2          jane    303  1.418938e-04
3        austen      3  1.404889e-06
4        volume     32  1.498548e-05
...         ...    ...           ...
41482  endowing      1  4.682963e-07
41483   delving      1  4.682963e-07
41484  germinal      1  4.682963e-07
41485   blither      1  4.682963e-07
41486  ushering      1  4.682963e-07

[41487 rows x 3 columns]


# **Noisy Channel Model Implementation**

*Again, you don't have to follow this directly, but consider placing your implementation of the model in the code block below.*

In [88]:
#Kyle's error trace
def calculate_edit_type_and_edit(input_word, candidate_word):
    if len(input_word) == len(candidate_word):
        # Check for substitution
        diff_count = 0
        edit_type = "sub"
        edit = ""
        for i in range(len(input_word)):
            if input_word[i] != candidate_word[i]:
                diff_count += 1
                edit = input_word[i] + "|"
                edit += candidate_word[i]
        if diff_count == 1:
            return edit_type, edit

    elif len(input_word) + 1 == len(candidate_word):
        # Check for insertion
        edit_type = "ins"
        edit = ""
        i, j = 0, 0
        if candidate_word[0:-1] == input_word:
            edit +=  input_word[-1] + "|" + candidate_word[-2] + candidate_word[-1]
            return edit_type, edit
        else:
            while i < len(input_word) and j < len(candidate_word):
                if input_word[i] != candidate_word[j]:
                    edit += input_word[j-1] + "|" + input_word[j-1] + candidate_word[j]
                    j += 1
                else:
                    i += 1
                    j += 1
            if j == len(candidate_word):
                return edit_type, edit

    elif len(input_word) - 1 == len(candidate_word):
        # Check for deletion
        edit_type = "del"
        edit = ""
        i, j = 0, 0
        if input_word[0:-1] == candidate_word:
            edit += input_word[-1] + input_word[-2] + "|" + candidate_word[-1]
            return edit_type, edit
        else:
            while i < len(input_word) and j < len(candidate_word):
                if input_word[i] != candidate_word[j]:
                    edit += input_word[i-1] + input_word[i] + "|" + candidate_word[i - 1]
                    i += 1
                else:
                    i += 1
                    j += 1
        if i == len(input_word):
            return edit_type, edit

    # Check for transposition
    if len(input_word) == len(candidate_word) and input_word != candidate_word:
        for i in range(len(input_word) - 1):
            if input_word[i] == candidate_word[i + 1] and input_word[i + 1] == candidate_word[i]:
                edit_type = "trans"
                edit = candidate_word[i:i+2] + "|" + input_word[i:i+2]
                return edit_type, edit

    return None, None  # No valid edit type found



In [98]:
%pip install editdistpy
from editdistpy import damerau_osa as ld
def spell_correct(input, all_words):
      if input in all_words:
          print(lang.index)
          return lang[lang['word']==input]
      d = 1
      candidate_list = []
      while not candidate_list:
          candidate_list = (w for w in all_words if ld.distance(input,w,d) > -1)
          d+=1
      probable = lang.loc[lang['word'].isin(candidate_list)].sort_values(['P'], ascending=False)
      return probable



def get_candidates(input,all_words,k) :
    def make_df(row):
        wordP = row['P']
        edit_type, edit = calculate_edit_type_and_edit(input, row['word'])
        try:
            editP = err.loc[err['error'] == edit,'P'].item()
        except:
            editP = 0
        return pd.Series({'Word' : input, 'candidate' : row['word'], 'edit_type' : edit_type, 'edit' : edit, 'P(c)' : wordP, 'P(w|c)' : editP, 'P(c) x P(w|c)' : (wordP * editP)})
    if  input in  all_words:
        print("Word is spelled correctly")
        return

    c = spell_correct(input, all_words)
    df = pd.DataFrame(columns=['Word', 'candidate', 'edit_type', 'edit', 'P(c)', 'P(w|c)', 'P(c) x P(w|c)'])
    temp = c.apply(make_df,axis = 1)

    df = pd.concat([df, temp])
    print(df.nlargest(k,'P(c) x P(w|c)'))


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [99]:
#main function
test_input = input("User Input: ").lower()

k = input("Maximum Candidates: ")
#get_candidates(test_input,all_words,k)
get_candidates(test_input,all_words,int(k))

User Input:  theeee
Maximum Candidates:  11


         Word candidate edit_type              edit          P(c)    P(w|c)  \
38469  theeee    theefe       sub               e|f  4.682963e-07  0.000051   
221    theeee     there       del      ee|eee|ree|e  2.987731e-03  0.000000   
10354  theeee      thee      None              None  2.251100e-03  0.000000   
146    theeee     these       del      ee|eee|see|e  1.334645e-03  0.000000   
472    theeee     three       del  he|hee|ree|eee|e  5.750679e-04  0.000000   
7862   theeee    thence      None              None  1.119228e-04  0.000000   
35105  theeee     heere       del      et|eee|ree|e  6.322001e-05  0.000000   
3138   theeee    cheese     trans             ee|ee  1.123911e-05  0.000000   
9038   theeee     theme       del      ee|eee|mee|e  8.429334e-06  0.000000   
38666  theeee    themes      None              None  5.151260e-06  0.000000   
21462  theeee    threes     trans             ee|ee  3.278074e-06  0.000000   

       P(c) x P(w|c)  
38469   2.397033e-11  
221  